# Introduction

Notebook para fazer análise de modelos e de resultados de experimentos feitos e armazenados no Neptune

# Import Libraries

In [13]:
import os
import sys
import neptune
import zipfile
import shutil
import numpy as np
import pandas as pd

from datetime import datetime

from tqdm.notebook import tqdm

if '..' not in sys.path:
    sys.path.insert(0, '..')

import config

# Download de Modelos

## Fetch Project

In [3]:
icao_nets_proj = neptune.init('guilhermemg/icao-nets-training', api_token=config.NEPTUNE_API_TOKEN)

## Fetch Experiments

In [3]:
df = icao_nets_proj.get_leaderboard(owner='guilhermemg')

In [4]:
df.to_csv('single_task_exps/experiments_data.csv', index=False)

## Select Experiments

In [5]:
def process_gt_names(gt_names):
    gt_names_dict = {'train_validation_test': gt_names.split(':')[-1]}
    train_val_test = gt_names_dict['train_validation_test']
    train_val_test = train_val_test[:-3]
    return train_val_test

In [7]:
df = pd.read_csv('single_task_exps/experiments_data.csv')
df = df[['id','property_icao_reqs','property_aligned','property_gt_names', 'created']]
df = df.dropna()
df = df[df.property_icao_reqs.str.contains('\[')]
df['property_gt_names'] = df.property_gt_names.apply(lambda row : process_gt_names(row))

init_date_exps = datetime(2021, 5, 12)
final_date_exps = datetime(2021, 5, 19)

df['created'] = df['created'].astype('datetime64[ns]')

df = df[(df.created >=  init_date_exps) & (df.created <  final_date_exps)]

display(df.head(5))
print(df.shape)

assert df.shape[0] == 92 # 92 experiments !!!

,id,property_icao_reqs,property_aligned,property_gt_names,created
122,IC-376,['mouth'],True,'fvc',2021-05-12 14:09:59.262
123,IC-377,['rotation'],True,'fvc',2021-05-12 14:25:49.929
124,IC-378,['l_away'],True,'fvc',2021-05-12 14:41:49.938
125,IC-379,['eyes_closed'],True,'fvc',2021-05-12 14:57:41.876
126,IC-380,['close'],True,'fvc',2021-05-12 15:13:22.753


(92, 5)


## Download Models

In [22]:
if '../../../../notebooks/' not in sys.path:
    sys.path.insert(0, '../../../../notebooks/')

from utils import constants as cts

In [21]:
def download_model(req, ds, align):
    print(f' Requisite: {req.upper()} | Dataset: {ds.upper()} | Aligned: {align}')
    
    exp_id = df[((df.property_icao_reqs.str.contains(req)) & df.property_aligned == align) & (df.property_gt_names.str.contains(ds))].id.values[0]
    print(f'.. Identified experiment id: {exp_id}')

    my_exp = icao_nets_proj.get_experiments(id=exp_id)[0]
    print('.. Downloading artifacts...')
    aligned = 'aligned' if align else 'not_aligned'
    dest_dir = f'../prev_trained_models/single_task/{ds}_{aligned}/{req}'
    my_exp.download_artifacts(destination_dir=dest_dir)
    print(' .. Download done!')
    
    with zipfile.ZipFile(os.path.join(dest_dir, 'output.zip'), 'r') as zip_ref:
        zip_ref.extractall(dest_dir)
    
    os.remove(os.path.join(dest_dir, 'output.zip'))
    shutil.move(os.path.join(dest_dir, 'output', 'variables'), dest_dir)
    shutil.move(os.path.join(dest_dir, 'output', 'saved_model.pb'), dest_dir)
    shutil.rmtree(os.path.join(dest_dir, 'output'))
    
    print('.. Folders set')
    print('-----------------------------')

In [24]:
reqs = cts.ICAO_REQ.list_reqs_names()
datasets = ['fvc', 'pybossa']
aligns = [True, False]

for req in reqs[-2:]:
    for ds in datasets:
        for alig in aligns:
            download_model(req, ds, alig)

 Requisite: BACKGROUND | Dataset: FVC | Aligned: True
.. Identified experiment id: IC-401
.. Downloading artifacts...
 .. Download done!
.. Folders set
-----------------------------
 Requisite: BACKGROUND | Dataset: FVC | Aligned: False
.. Identified experiment id: IC-376
.. Downloading artifacts...
 .. Download done!
.. Folders set
-----------------------------
 Requisite: BACKGROUND | Dataset: PYBOSSA | Aligned: True
.. Identified experiment id: IC-458
.. Downloading artifacts...
 .. Download done!
.. Folders set
-----------------------------
 Requisite: BACKGROUND | Dataset: PYBOSSA | Aligned: False
.. Identified experiment id: IC-412
.. Downloading artifacts...
 .. Download done!
.. Folders set
-----------------------------
 Requisite: RED_EYES | Dataset: FVC | Aligned: True
.. Identified experiment id: IC-405
.. Downloading artifacts...
 .. Download done!
.. Folders set
-----------------------------
 Requisite: RED_EYES | Dataset: FVC | Aligned: False
.. Identified experiment id: 